In [48]:
import numpy as np
import pandas as pd

In [49]:
def activation(x, deriv=False):
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [50]:
df=pd.read_csv("MIMOCompiled.csv", header=0)

In [51]:
df.head()

,target,h1real,h1imag,h2real,h2imag,y1real,y1imag,y2real,y2imag,n1real,...,ymod2real,ymod2img,ymod3real,ymod3img,ymod4real,ymod4img,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,0,0.380188,0.209425,1.296753,0.849082,1.683406,-1.002812,-0.727922,-0.006522,0.923215,...,0.609621,1.302034,-0.727922,-0.006522,0.372573,-1.747136,NaN,NaN,NaN,NaN
1,0,-1.597246,0.770869,0.609649,-0.253641,0.609621,1.302034,0.372573,-1.747136,-0.884864,...,0.609621,1.302034,-0.727922,-0.006522,0.372573,-1.747136,NaN,NaN,NaN,NaN
2,1,0.225401,-0.091873,-0.924675,0.518831,-0.092323,0.979530,-0.678165,0.607294,-0.525668,...,1.266874,0.063812,-0.678165,0.607294,-1.035015,0.060759,NaN,NaN,NaN,NaN
3,0,-0.306596,0.085087,0.242272,0.803508,1.266874,0.063812,-1.035015,0.060759,1.485875,...,1.266874,0.063812,-0.678165,0.607294,-1.035015,0.060759,NaN,NaN,NaN,NaN
4,1,2.530309,-0.485622,1.958288,0.333530,3.692287,-0.333293,-0.425962,-1.103060,1.378002,...,0.703411,0.909991,-0.425962,-1.103060,3.286375,0.475154,NaN,NaN,NaN,NaN


In [52]:
important_features=['h1real' ,'h1imag' ,'h2real' ,'h2imag' ,'y1real' ,'y1imag' ,'y2real' ,'y2imag' ,'n1real' ,'n1imag' ,'n2real' ,'n2imag']

In [88]:
def SplitTrainAndTest(numberOfTraining, df):
    msk1=np.random.rand(numberOfTraining)<1
    msk2=np.random.rand(len(df)-numberOfTraining)<0
    msk=np.concatenate((msk1,msk2), axis=0)
    train=df[msk]
    test=df[~msk]
    return train, test   

In [89]:
trainData, testData=SplitTrainAndTest(800, df)

In [90]:
X_train=trainData[important_features].values
y_train=trainData['target'].values
X_test=testData[important_features].values
y_test=testData['target'].values

In [91]:
X=X_train
y=y_train
num_examples = len(X) # training set size
nn_input_dim = 12 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = 0.01 # learning rate for gradient descent
reg_lambda = 0.01 # regularization strength
# Helper function to evaluate the total loss on the dataset
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation to calculate our predictions
    z1 = X.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # Calculating the loss
    corect_logprobs = -np.log(probs[range(num_examples), y])
    data_loss = np.sum(corect_logprobs)
    # Add regulatization term to loss (optional)
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) + np.sum(np.square(W2)))
    return 1./num_examples * data_loss
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(W2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return np.argmax(probs, axis=1)
# This function learns parameters for the neural network and returns the model.
# - nn_hdim: Number of nodes in the hidden layer
# - num_passes: Number of passes through the training data for gradient descent
# - print_loss: If True, print the loss every 1000 iterations
def build_model(nn_hdim, num_passes=200, print_loss=False):   
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))

    # This is what we return at the end
    model = {}
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):

        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
          print "Loss after iteration %i: %f" %(i, calculate_loss(model))
    
    return model

In [92]:
model1=build_model_with_stopping_criteria(60,desired_loss=0.01, print_loss=True)
print "--------Model 1 Trained Successfully !----------"
prediction1=predict(model1, X_test)
print "--------Accuracy For Model 1----------"
get_accuracy(prediction1, y_test)
print "----------------------------------"

Loss after iteration 100: 3.059621
Loss after iteration 200: 0.237820
Loss after iteration 300: 0.040986
Loss after iteration 400: 0.039630
Loss after iteration 500: 0.038714
Loss after iteration 600: 0.037938
Loss after iteration 700: 0.037230
Loss after iteration 800: 0.036564
Loss after iteration 900: 0.035927
Loss after iteration 1000: 0.035313
Loss after iteration 1100: 0.034717
Loss after iteration 1200: 0.034138
Loss after iteration 1300: 0.033574
Loss after iteration 1400: 0.033024
Loss after iteration 1500: 0.032488
Loss after iteration 1600: 0.031963
Loss after iteration 1700: 0.031451
Loss after iteration 1800: 0.030951
Loss after iteration 1900: 0.030462
Loss after iteration 2000: 0.029984
Loss after iteration 2100: 0.029517
Loss after iteration 2200: 0.029060
Loss after iteration 2300: 0.028614
Loss after iteration 2400: 0.028178
Loss after iteration 2500: 0.027752
Loss after iteration 2600: 0.027335
Loss after iteration 2700: 0.026926
Loss after iteration 2800: 0.026526
L

In [61]:
def get_accuracy(prediction, target):
    num_correct=(prediction==target).sum()
    print num_correct
    accuracy=1.0*num_correct/len(target)
    return accuracy

In [62]:
prediction1=predict(model1,X_test)
prediction2=predict(model2,X_test)
prediction3=predict(model3,X_test)

In [63]:
get_accuracy(prediction1, y_test)

110


0.5472636815920398

In [64]:
get_accuracy(prediction2, y_test)
get_accuracy(prediction3, y_test)

102
124


0.6169154228855721

In [76]:
def build_model_with_stopping_criteria(nn_hdim, desired_loss=0.01, print_loss=False):   
    # Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)
    b1 = np.zeros((1, nn_hdim))
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)
    b2 = np.zeros((1, nn_output_dim))
    model_loss=1

    # This is what we return at the end
    model = {}
    i=0
    # Gradient descent. For each batch...
    while model_loss>desired_loss:
        i+=1

        # Forward propagation
        z1 = X.dot(W1) + b1
        a1 = np.tanh(z1)
        z2 = a1.dot(W2) + b2
        exp_scores = np.exp(z2)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta3 = probs
        delta3[range(num_examples), y] -= 1
        dW2 = (a1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(W2.T) * (1 - np.power(a1, 2))
        dW1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Add regularization terms (b1 and b2 don't have regularization terms)
        dW2 += reg_lambda * W2
        dW1 += reg_lambda * W1

        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        model_loss=calculate_loss(model)
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 100 == 0:
          print "Loss after iteration %i: %f" %(i, calculate_loss(model))
    
    return model

In [77]:
model31=build_model_with_stopping_criteria(60,desired_loss=0.01, print_loss=True)

Loss after iteration 100: 4.631361
Loss after iteration 200: 0.054485
Loss after iteration 300: 0.048081
Loss after iteration 400: 0.046809
Loss after iteration 500: 0.045811
Loss after iteration 600: 0.044907
Loss after iteration 700: 0.044055
Loss after iteration 800: 0.043238
Loss after iteration 900: 0.042448
Loss after iteration 1000: 0.041682
Loss after iteration 1100: 0.040936
Loss after iteration 1200: 0.040208
Loss after iteration 1300: 0.039499
Loss after iteration 1400: 0.038806
Loss after iteration 1500: 0.038130
Loss after iteration 1600: 0.037469
Loss after iteration 1700: 0.036823
Loss after iteration 1800: 0.036192
Loss after iteration 1900: 0.035575
Loss after iteration 2000: 0.034972
Loss after iteration 2100: 0.034381
Loss after iteration 2200: 0.033804
Loss after iteration 2300: 0.033239
Loss after iteration 2400: 0.032687
Loss after iteration 2500: 0.032146
Loss after iteration 2600: 0.031617
Loss after iteration 2700: 0.031099
Loss after iteration 2800: 0.030593
L

In [78]:
prediction31=predict(model31, X_test)

In [80]:
get_accuracy(prediction31, y_test)

109


0.5422885572139303

In [93]:
model32=build_model_with_stopping_criteria(60,desired_loss=0.01, print_loss=True)
prediction32=predict(model32, X_test)
get_accuracy(prediction32, y_test)

Loss after iteration 100: 2.642174
Loss after iteration 200: 0.054657
Loss after iteration 300: 0.039668
Loss after iteration 400: 0.037906
Loss after iteration 500: 0.036923
Loss after iteration 600: 0.036145
Loss after iteration 700: 0.035455
Loss after iteration 800: 0.034815
Loss after iteration 900: 0.034209
Loss after iteration 1000: 0.033629
Loss after iteration 1100: 0.033070
Loss after iteration 1200: 0.032528
Loss after iteration 1300: 0.032001
Loss after iteration 1400: 0.031488
Loss after iteration 1500: 0.030988
Loss after iteration 1600: 0.030500
Loss after iteration 1700: 0.030022
Loss after iteration 1800: 0.029556
Loss after iteration 1900: 0.029100
Loss after iteration 2000: 0.028654
Loss after iteration 2100: 0.028218
Loss after iteration 2200: 0.027792
Loss after iteration 2300: 0.027375
Loss after iteration 2400: 0.026967
Loss after iteration 2500: 0.026568
Loss after iteration 2600: 0.026177
Loss after iteration 2700: 0.025795
Loss after iteration 2800: 0.025421
L

0.56

In [82]:
prediction32=predict(model32, X_test)

In [83]:
get_accuracy(prediction32, y_test)

125


0.6218905472636815

In [85]:
model33=build_model_with_stopping_criteria(55,desired_loss=0.01, print_loss=True)
prediction33=predict(model33, X_test)
get_accuracy(prediction33, y_test)

Loss after iteration 100: 3.274432
Loss after iteration 200: 0.840677
Loss after iteration 300: 0.037666
Loss after iteration 400: 0.035806
Loss after iteration 500: 0.034765
Loss after iteration 600: 0.033974
Loss after iteration 700: 0.033295
Loss after iteration 800: 0.032680
Loss after iteration 900: 0.032106
Loss after iteration 1000: 0.031561
Loss after iteration 1100: 0.031039
Loss after iteration 1200: 0.030535
Loss after iteration 1300: 0.030047
Loss after iteration 1400: 0.029572
Loss after iteration 1500: 0.029108
Loss after iteration 1600: 0.028656
Loss after iteration 1700: 0.028215
Loss after iteration 1800: 0.027783
Loss after iteration 1900: 0.027360
Loss after iteration 2000: 0.026947
Loss after iteration 2100: 0.026542
Loss after iteration 2200: 0.026147
Loss after iteration 2300: 0.025759
Loss after iteration 2400: 0.025380
Loss after iteration 2500: 0.025009
Loss after iteration 2600: 0.024645
Loss after iteration 2700: 0.024289
Loss after iteration 2800: 0.023940
L

0.4925373134328358